# 实验一
自定义VGG16网络，并在Kaggle猫/狗数据集上进行训练和测试

## 1.加载keras模块

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras import backend as K
import numpy as np
import PIL

Using TensorFlow backend.


### 定义CNN网络结构



In [2]:
img_width, img_height = 150, 150
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
 #定义VGG16结构
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=input_shape))#卷积输入层，指定了输入图像的大小
model.add(Conv2D(64, 3, 3, activation='relu'))#64个3x3的卷积核，生成64*224*224的图像，激活函数为relu
model.add(ZeroPadding2D((1,1)))#补0，保证图像卷积后图像大小不变，其实用padding = 'valid'参数就可以了
model.add(Conv2D(64, 3, 3, activation='relu'))#再来一次卷积 生成64*224*224
model.add(MaxPooling2D((2,2), 2))#pooling操作，相当于变成64*112*112

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), 2))#128*56*56

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), 2))#256*28*28

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), 2))#512*14*14

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, 3, 3, activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, 3, 3, activation='relu'))
model.add(MaxPooling2D((2,2), 2))#到这里已经变成了512*7*7

model.add(Flatten())#压平上述向量，变成一维25088
model.add(Dense(4096, activation='relu'))#全连接层有4096个神经核，参数个数就是4096*25088
model.add(Dropout(0.5))#0.5的概率抛弃一些连接
model.add(Dense(4096, activation='relu'))#再来一个全连接
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

#在compile函数里定义loss函数、优化器和metrics
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

d:\programdata\anaconda3\envs\tensorflow2.0\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
d:\programdata\anaconda3\envs\tensorflow2.0\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  if sys.path[0] == '':
d:\programdata\anaconda3\envs\tensorflow2.0\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  app.launch_new_instance()
d:\programdata\anaconda3\envs\tensorflow2.0\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
d:\programdata\anaconda3\envs\tensorflow2.0\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 AP

### 查看model架构



In [3]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 152, 152, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 64)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 152, 152, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 150, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 77, 77, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 75, 128)      

### 定义ImageDataGenerator


In [4]:
train_data_dir = (r'C:\Users\Administrator\Desktop\dogs-vs-cats\train')
validation_data_dir = (r'C:\Users\Administrator\Desktop\dogs-vs-cats\validation')
nb_train_samples = 10835
nb_validation_samples = 4000
epochs = 1
batch_size = 20

#定义ImageDataGenerator为生成training和validation samples
train_datagen = ImageDataGenerator(
        #执行其他处理前乘到整个图像上
        rescale=1./255,
        #随机错切换角度
        shear_range=0.2,
        #随机缩放范围
        zoom_range=0.2,
        ## 一半图像水平翻转
        horizontal_flip=True)
#声明ImageDataGenerator类型的test_datagen用于对validation图像进行像素归一化
test_datagen = ImageDataGenerator(rescale=1. / 255)
#调用flow_from_directory生成train_generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')
#调用flow_from_directory生成validation_generator
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')



Found 10835 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


### 训练模型



In [28]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/1
  8/541 [..............................] - ETA: 1:13:25 - loss: 7.0916 - accuracy: 0.5375

KeyboardInterrupt: 

### 使用训练后模型预测图像





In [ ]:
import numpy as np
import cv2
img_width, img_height = 150,150
img = cv2.resize(cv2.imread(r'C:\Users\Administrator\Desktop\dogs-vs-cats\test\7.jpg'), (img_width, img_height)).astype(np.float32)

model = VGG16(weights='imagenet',include_top=False)

x = img_to_array(img)

x = np.expand_dims(x, axis=0)

#x = preprocess_input(x)

features = model.predict(x)

print(features)